In [1]:
import numpy as np
import torch

# NumPy -> Torch shares memory (CPU only)
np_array = np.arange(1.0, 8.0)          # dtype=float64 by default
t_shared = torch.from_numpy(np_array)   # shares memory, keeps dtype

print("NumPy dtype:", np_array.dtype)       # float64
print("Tensor dtype:", t_shared.dtype)      # torch.float64

# Convert to float32 explicitly (makes a NEW tensor, breaks sharing)
t32 = t_shared.to(torch.float32)  # or: t_shared.float()
print("Tensor -> float32 dtype:", t32.dtype)  # torch.float32

NumPy dtype: float64
Tensor dtype: torch.float64
Tensor -> float32 dtype: torch.float32


In [2]:
# Out-of-place change: new array object; tensor still points to old memory
np_array = np_array + 1
print("Tensor after np_array = np_array + 1 (out-of-place):", t_shared)  # unchanged

# In-place change: affects the shared tensor
np_array2 = np.arange(1.0, 8.0)
t_shared2 = torch.from_numpy(np_array2)
np_array2 += 10  # in-place
print("Tensor after np_array2 += 10 (in-place):", t_shared2)  # reflects +10

Tensor after np_array = np_array + 1 (out-of-place): tensor([1., 2., 3., 4., 5., 6., 7.], dtype=torch.float64)
Tensor after np_array2 += 10 (in-place): tensor([11., 12., 13., 14., 15., 16., 17.], dtype=torch.float64)


In [3]:
t = torch.ones(7)        # CPU float32 by default
np_view = t.numpy()      # shares memory with t (CPU only)

print("Tensor:", t)
print("NumPy view:", np_view, np_view.dtype)  # float32

# In-place operation on tensor reflects in NumPy view
t.add_(1)   # in-place
print("After t.add_(1):", t, np_view)

# Out-of-place reassign breaks the link
t = t + 1
print("After t = t + 1:", t)      # 3s
print("NumPy view still:", np_view)  # still 2s

Tensor: tensor([1., 1., 1., 1., 1., 1., 1.])
NumPy view: [1. 1. 1. 1. 1. 1. 1.] float32
After t.add_(1): tensor([2., 2., 2., 2., 2., 2., 2.]) [2. 2. 2. 2. 2. 2. 2.]
After t = t + 1: tensor([3., 3., 3., 3., 3., 3., 3.])
NumPy view still: [2. 2. 2. 2. 2. 2. 2.]


In [ ]:
# Suppose t_gpu lives on CUDA:
t_gpu = torch.ones(7, device="cuda")
# Move to CPU and detach before NumPy:
np_from_cuda = t_gpu.detach().cpu().numpy()

In [5]:
import os
import random
import numpy as np
import torch

def set_seed(seed: int = 42, deterministic: bool = False, cudnn_benchmark: bool = False):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.benchmark = cudnn_benchmark
    torch.backends.cudnn.deterministic = deterministic

    if deterministic:
        try:
            torch.use_deterministic_algorithms(True)
        except Exception as e:
            print(f"Could not enforce strict deterministic mode: {e}")

    print(f"[Seed Set] seed={seed}, deterministic={deterministic}, cudnn.benchmark={cudnn_benchmark}")

# Example usage
set_seed(42, deterministic=True, cudnn_benchmark=False)

# Show that seeding repeats values
torch.manual_seed(42)
A = torch.rand(3, 4)
torch.manual_seed(42)
B = torch.rand(3, 4)
print("A equals B everywhere?", torch.equal(A, B))
print(A)
print(B)

[Seed Set] seed=42, deterministic=True, cudnn.benchmark=False
A equals B everywhere? True
tensor([[0.8823, 0.9150, 0.3829, 0.9593],
        [0.3904, 0.6009, 0.2566, 0.7936],
        [0.9408, 0.1332, 0.9346, 0.5936]])
tensor([[0.8823, 0.9150, 0.3829, 0.9593],
        [0.3904, 0.6009, 0.2566, 0.7936],
        [0.9408, 0.1332, 0.9346, 0.5936]])


In [7]:
import numpy as np, torch, random

# reproducibility
def seed_everything(seed=123):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    try: torch.use_deterministic_algorithms(True)
    except: pass

seed_everything(123)

# numpy -> torch
np_a = np.arange(1.0, 6.0)                 # float64
t_view = torch.from_numpy(np_a)            # shares memory
t_copy_f32 = t_view.float()                # breaks sharing, float32

# sharing check
np_a += 100                                # in-place -> updates t_view
print("Shared tensor after np in-place:", t_view)

# torch -> numpy
t = torch.ones(5)                           # CPU float32
np_view = t.numpy()                         # shares memory
t.add_(5)
print("Tensor & NumPy after t.add_(5):", t, np_view)

# equality w/ seeding
torch.manual_seed(123); X = torch.rand(2,3)
torch.manual_seed(123); Y = torch.rand(2,3)
print("X == Y everywhere?", torch.equal(X, Y))

Shared tensor after np in-place: tensor([101., 102., 103., 104., 105.], dtype=torch.float64)
Tensor & NumPy after t.add_(5): tensor([6., 6., 6., 6., 6.]) [6. 6. 6. 6. 6.]
X == Y everywhere? True
